<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/plot_aviso_fields_at_min_SLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plot AVISO:
- Geostrophic currents
- ADT
- SLA

## At the location and date of minimum sea level pressure
- If the min SLP was sustrained for more than one time instance, we plot the first occurrence.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_high_inten(data_path, hurr_name, loc_to_save, dLat=10, dLon=10):

    track_fName = "{}/{}/{}_{}_{}.nc".format(data_path, str(year), str(year), myBasin, hurr_name)
    aviso_fName = "{}/{}/{}_{}_{}_AVISO_L4.nc".format(data_path, str(year), str(year), myBasin, hurr_name)
    #print("{}\n{}".format(track_fName, aviso_fName))
    ds_track = xr.open_dataset(track_fName)
    ds_aviso = xr.open_dataset(aviso_fName)

    U = np.sqrt(ds_aviso.ugos**2+ds_aviso.vgos**2)

    # Date of minimum MSLP/highest intensity
    tMin=ds_track.mslp.where(ds_track.mslp==ds_track.mslp.min(), drop=True).time.values[0]
    print("Min MSLP at: {}".format(tMin))
    latMin, lonMin = [ds_track.sel(time=tMin).lat.values, ds_track.sel(time=tMin).lon.values]
    print("Min SLP at: [lat, lon]=\t{}\t{}".format(latMin, lonMin))

    lat_start, lat_end = latMin-dLat, latMin+dLat
    lon_start, lon_end = lonMin-dLon, lonMin+dLon
    #print(lat_start, lat_end, lon_start, lon_end)

    fig=plt.figure(figsize=(16, 4))

    ax1=plt.subplot(131)
    U.sel(time=tMin, method="nearest").sel(latitude=slice(lat_start, lat_end), longitude=slice(lon_start, lon_end)).\
    plot(ax=ax1, cmap=plt.cm.Blues, add_colorbar=True, cbar_kwargs={'pad':0.01, 'label':"Geostrophic current speed [m/s]"})
    plt.scatter(ds_track.lon, ds_track.lat, s=ds_track.vmax/5., c='g', marker='o', edgecolor=None)
    plt.plot(lonMin, latMin, 'g*', ms=12), plt.title("") #plt.title("{}".format(tMin))
    plt.text(lon_start, lat_end-1, "(a)")

    ax2=plt.subplot(132)
    ds_aviso.adt.sel(time=tMin, method="nearest").sel(latitude=slice(lat_start, lat_end), longitude=slice(lon_start, lon_end)).\
    plot(ax=ax2, cmap=plt.cm.Spectral_r, add_colorbar=True, cbar_kwargs={'pad':0.01, 'label':"ADT [m]"})
    plt.scatter(ds_track.lon, ds_track.lat, s=ds_track.vmax/5., c='g', marker='o', edgecolor=None)
    plt.plot(lonMin, latMin, 'g*', ms=12), plt.title("")
    ax2.set_ylabel("")
    plt.text(lon_start, lat_end-1, "(b)")

    ax3=plt.subplot(133)
    ds_aviso.sla.sel(time=tMin, method="nearest").sel(latitude=slice(lat_start, lat_end), longitude=slice(lon_start, lon_end)).\
    plot(ax=ax3, cmap=plt.cm.bwr, add_colorbar=True, cbar_kwargs={'pad':0.01, 'label':"SLA [m]"})
    plt.scatter(ds_track.lon, ds_track.lat, s=ds_track.vmax/5., c='g', marker='o', edgecolor=None)
    plt.plot(lonMin, latMin, 'g*', ms=12), plt.title("")
    ax3.set_ylabel("")
    plt.text(lon_start, lat_end-1, "(c)")

    plt.tight_layout()

    figName= loc_to_save + 'high_intensity_{}_{}.png'.format(str(year), hurr_name)
    plt.savefig(figName, dpi=120)
    plt.close("all")

    del ds_track, ds_aviso

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

## User inputs

In [ ]:
year=2024
myBasin='north_atlantic'
cat_threshold = 4

data_path = '/content/drive/MyDrive/datasets/hurr/'

## Plot all (major) storms of chosen year

In [ ]:
track_summ_fName = data_path + '{}/'.format(year) + 'hurdat2_{}_{}.csv'.format(myBasin, year)
print("Reading {} summary data from:\n{}".format(year, track_summ_fName))

season_data=pd.read_csv(track_summ_fName)
major_hurr_names = season_data['name'][season_data['category'] >=cat_threshold]

for hurr_name in major_hurr_names:
  print(hurr_name)
  plot_high_inten(data_path, hurr_name, loc_to_save=data_path, dLat=10, dLon=10)